# Mass-Spring System Simulators

These simulations use Taichi. To run, first run the code cell below to import taichi

In [2]:
import taichi as ti

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, osx, python 3.9.7
[I 09/21/22 13:03:02.329 934842] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout


# Explicit Euler integration scheme

### 2 particle system with damping. 

Each particle's mass can be controlled individually using the keyboard

In [71]:
import taichi as ti

ti.init()

num_particles = 2
dt = 1e-3
substeps = 10

spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=num_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]
            
            
    # print(v[0], v[1])
    # print(f[0], f[1])
    # print("")
    

def main():
    gui = ti.GUI('Two Springs with damping', (640, 480))
    
    # initialize positions of particles
    x[0][0] = 0.7 # left x
    x[0][1] = 0.5 # left y
    x[1][0] = 0.3 # right x
    x[1][1] = 0.5 # right y
    # set initial velocities to 0
    v[0][0] = 0
    v[0][1] = 0
    v[1][0] = 0
    v[1][1] = 0
    
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(num_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'a': # lower left mass 
                mass[0] /= 1.1
            elif e.key == 'd': # increase left mass
                mass[0] *= 1.1
            elif e.key == 's': # lower right mass
                mass[1] /= 1.1
            elif e.key == 'w': # increase right mass
                mass[1] *= 1.1
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw spring
        X = x.to_numpy()
        gui.line(begin=X[0], end=X[1], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Press a to decrease the left mass, d to increase it',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press s to decrease the right mass, w to increase it',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'Left Mass {mass[0]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Right Mass {mass[1]:.1f}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()



[Taichi] Starting on arch=x64


### Multi particle system (fully connected)

Uses keyboard input to add new particles with a variable spring constant. Damping is deactivated for this, but can be added by uncommenting line 35

In [18]:
import taichi as ti

ti.init()

max_particles = 10
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10

spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=max_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles[None]):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles[None]):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                # f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles[None]):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]

        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Multi Particle Mass-Spring System', (640, 480))
    
    new_particle(0.3, 0.5)
    
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(max_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'a': # lower left mass 
                mass[0] /= 1.1
            elif e.key == 'd': # increase left mass
                mass[0] *= 1.1
            elif e.key == 's': # lower right mass
                mass[1] /= 1.1
            elif e.key == 'w': # increase right mass
                mass[1] *= 1.1
            elif e.key == ti.GUI.UP:
                spring_k[None] *= 1.1
            elif e.key == ti.GUI.DOWN:
                spring_k[None] /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw springs
        X = x.to_numpy()
        for i in range(num_particles[None]):
            for j in range(num_particles[None]):
                if i != j:
                    gui.line(begin=X[i], end=X[j], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press up/down arrows to change spring constant',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'k {spring_k[None]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Num particles {num_particles[None]}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


# Implicit Euler Integration Scheme

Credit to professor Bo Zhu, on whose COSC89 Course Notes the following code is based

### Two particle system

In [32]:
import numpy as np
import taichi as ti


@ti.data_oriented
class System:
    def __init__(self, NV):
        # this is set up for a TWO PARTICLE SYSTEM
        self.NV = NV # num verticles
        self.NE = NV - 1  # num edges, assuming only a two particle system
        self.x = ti.Vector.field(2, ti.f32, shape=self.NV)
        self.initPos = ti.Vector.field(2, ti.f32, shape=self.NV) # do I need this
        self.v = ti.Vector.field(2, dtype=ti.f32, shape=self.NV)
        self.f = ti.Vector.field(2, dtype=ti.f32, shape=self.NV)
        self.mass = ti.field(ti.f32, shape=self.NV)
        
        # for 2 particle system, comps will always be spring[0][0] and spring[0][1]
        self.spring = ti.Vector.field(2, ti.i32, self.NE) # each spring is held here where spring[num][0] is i and spring[num][1] is j

        #self.indices = ti.field(ti.i32, 2* self.NE)
        
        self.Jx = ti.Matrix.field(2, 2, ti.f32, shape=self.NE) # Jacobian wrt pos
        self.Jv = ti.Matrix.field(2, 2, ti.f32, shape=self.NE) # Jacobian wrt vel
        
        self.rest_length = ti.field(dtype=ti.f32, shape=self.NE)
        self.ks = 10.0 # spring constant
        self.kd = 0.3 #damping constant
        
        # self.gravity = ti.Vector([0.0, -2.0])
        self.init_pos()
        self.init_edges()
        
        self.MassBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV, 2 * self.NV, max_num_triplets = 10000)
        self.DBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV, 2 * self.NV, max_num_triplets = 10000)
        self.KBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV, 2 * self.NV, max_num_triplets = 10000)
        
        self.init_mass_sp(self.MassBuilder)
        self.M = self.MassBuilder.build()
        
    
    @ti.kernel
    def init_pos(self):
        self.x[0] = ti.Vector([0.3, 0.5])
        self.x[1] = ti.Vector([0.7, 0.5])
        self.v[0] = ti.Vector([0, 0])
        self.v[1] = ti.Vector([0, 0])
        self.mass[0] = 0.5
        self.mass[1] = 0.5
    
    
    @ti.kernel
    def init_edges(self):
        self.spring[0] = ti.Vector([0, 1])
        self.rest_length[0] = 0.2
    
    @ti.kernel
    def init_mass_sp(self, M: ti.types.sparse_matrix_builder()):
        for i in range(self.NV):
            mass = self.mass[i]
            M[2 * i, 2 * i] += mass
            M[2 * i + 1, 2 * i + 1] += mass

            
    @ti.func
    def clear_force(self):
        for i in self.f:
            self.f[i] = ti.Vector([0.0, 0.0])
    
    
    @ti.kernel
    def compute_force(self):
        self.clear_force()
        # for i in self.f:
            # self.f += self.gravity * self.mass[i]
        
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            pos1, pos2 = self.x[idx1], self.x[idx2]
            x_ji = pos2 - pos1
            d = x_ji.norm() # distance
            force = self.ks * (d - self.rest_length[i]) * x_ji.normalized()
            self.f[idx1] += force
            self.f[idx2] -= force
        
    
    @ti.kernel
    def compute_Jacobians(self):
        pass
    
    
    @ti.kernel
    def assemble_D(self, D: ti.types.sparse_matrix_builder()):
        pass
    
    
    @ti.kernel
    def assemble_K(self, K: ti.types.sparse_matrix_builder()):
        pass

    
    @ti.kernel
    def updatePosVel(self, dt: ti.f32, dv: ti.types.ndarray()):
        for i in self.x:
            self.v[i] += ti.Vector([dv[2*i], dv[2*i + 1]])
            self.x[i] += dt * self.v[i]
        
    
    # find new values for everything at this new time step
    def update(self, dt):
        self.compute_force()
        
        self.compute_Jacobians()
        
        self.assemble_D(self.DBuilder)
        D = self.DBuilder.build()
        
        self.assemble_K(self.KBuilder)
        K = self.KBuilder.build()
        
        A = self.M - dt * D - dt**2 * K
        vel = self.v.to_numpy().reshape(2 * self.NV)
        force = self.f.to_numpy().reshape(2 * self.NV)
        
        
        # do I need to reshape v and f?
        b = self.M @ vel + dt * force - dt * D @ vel
        
        # solve sparse system
        solver = ti.linalg.SparseSolver() # what solver type do I use here?
        solver.analyze_pattern(A)
        solver.factorize(A)
        dv = solver.solve(b)
        self.updatePosVel(dt, dv)
        
        
    # draw everything
    def display(self, gui, radius=5, color=0xffffff):
        lines = self.spring
        pos = self.x
        edgeBegin = np.zeros(shape=(lines.shape[0], 2))
        edgeEnd = np.zeros(shape=(lines.shape[0], 2))
        for i in range(lines.shape[0]):
            idx1, idx2 = lines[i][0], lines[i][1]
            edgeBegin[i] = pos[idx1]
            edgeEnd[i] = pos[idx2]
        gui.lines(edgeBegin, edgeEnd, radius=2, color=0x0000ff)
        gui.circles(self.x.to_numpy(), radius, color)
    

def main():
    ti.init(arch=ti.cpu)
    dt = 0.01
    system = System(NV=2)

    gui = ti.GUI('Implicit Mass Spring System', res=(500, 500))
    while gui.running:
        for e in gui.get_events():
            if e.key == gui.ESCAPE:
                gui.running = False

        system.update(dt)

        system.display(gui)
        gui.show()

if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64


### Interacting with boundary

In [55]:
import argparse
import numpy as np
import taichi as ti


@ti.data_oriented
class Cloth:
    def __init__(self, N):
        self.N = N
        self.NF = 2 * N**2  # number of faces
        self.NV = (N + 1)**2  # number of vertices
        self.NE = 2 * N * (N + 1) + 2 * N * N  # numbser of edges
        self.pos = ti.Vector.field(2, ti.f32, self.NV)
        self.initPos = ti.Vector.field(2, ti.f32, self.NV)
        self.vel = ti.Vector.field(2, ti.f32, self.NV)
        self.force = ti.Vector.field(2, ti.f32, self.NV)
        self.mass = ti.field(ti.f32, self.NV)

        self.spring = ti.Vector.field(2, ti.i32, self.NE)
        self.indices = ti.field(ti.i32, 2 * self.NE)
        self.Jx = ti.Matrix.field(2, 2, ti.f32,
                                  self.NE)  # Jacobian with respect to position
        self.Jv = ti.Matrix.field(2, 2, ti.f32,
                                  self.NE)  # Jacobian with respect to velocity
        self.rest_len = ti.field(ti.f32, self.NE)
        self.ks = 1000.0  # spring stiffness
        self.kd = 0.5  # damping constant
        self.kf = 1.0e5  # fix point stiffness

        self.gravity = ti.Vector([0.0, -2.0])
        self.init_pos()
        self.init_edges()
        self.MassBuilder = ti.linalg.SparseMatrixBuilder(
            2 * self.NV, 2 * self.NV, max_num_triplets=10000)
        self.DBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV,
                                                      2 * self.NV,
                                                      max_num_triplets=10000)
        self.KBuilder = ti.linalg.SparseMatrixBuilder(2 * self.NV,
                                                      2 * self.NV,
                                                      max_num_triplets=10000)
        self.init_mass_sp(self.MassBuilder)
        self.M = self.MassBuilder.build()
        self.fix_vertex = [self.N, self.NV - 1]

    @ti.kernel
    def init_pos(self):
        # add initial velocity
        # vx_0 = (ti.random()-0.5)*0.8 -0.5
        vx_0 = -0.5
        for i, j in ti.ndrange(self.N + 1, self.N + 1):
            k = i * (self.N + 1) + j
            self.pos[k] = ti.Vector([i, j]) / self.N * 0.5 + ti.Vector(
                [0.25, 0.25])
            self.initPos[k] = self.pos[k]
            self.vel[k] = ti.Vector([vx_0 + (ti.random()-0.5)*0.8, 0])
            #self.vel[k] = ti.Vector([0, 0])
            self.mass[k] = 1.0

    @ti.kernel
    def init_edges(self):
        pos, spring, N, rest_len = ti.static(self.pos, self.spring, self.N,
                                             self.rest_len)
        for i, j in ti.ndrange(N + 1, N):
            idx, idx1 = i * N + j, i * (N + 1) + j
            spring[idx] = ti.Vector([idx1, idx1 + 1])
            rest_len[idx] = (pos[idx1] - pos[idx1 + 1]).norm()
        start = N * (N + 1)
        for i, j in ti.ndrange(N, N + 1):
            idx, idx1, idx2 = start + i + j * N, i * (N + 1) + j, i * (
                N + 1) + j + N + 1
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()
        start = 2 * N * (N + 1)
        for i, j in ti.ndrange(N, N):
            idx, idx1, idx2 = start + i * N + j, i * (N + 1) + j, (i + 1) * (
                N + 1) + j + 1
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()
        start = 2 * N * (N + 1) + N * N
        for i, j in ti.ndrange(N, N):
            idx, idx1, idx2 = start + i * N + j, i * (N + 1) + j + 1, (
                i + 1) * (N + 1) + j
            spring[idx] = ti.Vector([idx1, idx2])
            rest_len[idx] = (pos[idx1] - pos[idx2]).norm()

    @ti.kernel
    def init_mass_sp(self, M: ti.types.sparse_matrix_builder()):
        for i in range(self.NV):
            mass = self.mass[i]
            M[2 * i + 0, 2 * i + 0] += mass
            M[2 * i + 1, 2 * i + 1] += mass

    @ti.func
    def clear_force(self):
        for i in self.force:
            self.force[i] = ti.Vector([0.0, 0.0])

    
    @ti.func
    def check_boundaries(self):
        ### treating circles as point masses ###
        z = ti.Vector([0.0, 0.0])
            
        for idx in ti.ndrange(self.NV):
            # if the particle collides with lower boundary
            if self.pos[idx][1] < 0: 
                phi = ti.Vector([0, 1.0])
                dis_y = 0 - self.pos[idx][1]
                dis = ti.Vector([0, dis_y])
                # dis = ti.Vector([0,0]) - self.pos[idx][1]
                force_spring = self.ks * (dis.norm()) * phi
                force_damp = self.kd * ((z - self.vel[idx]).dot(phi)) * phi
                
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # if it collides with upper boundary
            elif self.pos[idx][1] > 1:
                phi = ti.Vector([0, -1.0])
                dis_y = 1 - self.pos[idx][1]
                dis = ti.Vector([0, dis_y])
                force_spring = self.ks * (dis.norm()) * phi
                force_damp = self.kd * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # left boundary
            if self.pos[idx][0] < 0:
                phi = ti.Vector([1.0, 0])
                dis_x = 0 - self.pos[idx][0]
                dis = ti.Vector([dis_x, 0])
                force_spring = self.ks * (dis.norm()) * phi
                force_damp = self.kd * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
            # right boundary
            elif self.pos[idx][0] > 1:
                phi = ti.Vector([-1.0, 0])
                dis_x = 1 - self.pos[idx][0]
                dis = ti.Vector([dis_x, 0])
                force_spring = self.ks * (dis.norm()) * phi
                force_damp = self.kd * ((z - self.vel[idx]).dot(phi)) * phi
                self.force[idx] += force_spring
                self.force[idx] -= force_damp
    
    @ti.kernel
    def compute_force(self):
        self.clear_force()
        for i in self.force:
            self.force[i] += self.gravity * self.mass[i]

        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            pos1, pos2 = self.pos[idx1], self.pos[idx2]
            dis = pos2 - pos1
            force = self.ks * (dis.norm() -
                               self.rest_len[i]) * dis.normalized()
            self.force[idx1] += force
            self.force[idx2] -= force
        self.check_boundaries()
    

    @ti.kernel
    def compute_Jacobians(self):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            pos1, pos2 = self.pos[idx1], self.pos[idx2]
            dx = pos1 - pos2
            I = ti.Matrix([[1.0, 0.0], [0.0, 1.0]])
            dxtdx = ti.Matrix([[dx[0] * dx[0], dx[0] * dx[1]],
                               [dx[1] * dx[0], dx[1] * dx[1]]])
            l = dx.norm()
            if l != 0.0:
                l = 1.0 / l
            self.Jx[i] = (I - self.rest_len[i] * l *
                          (I - dxtdx * l**2)) * self.ks
            self.Jv[i] = self.kd * I
            

    @ti.kernel
    def assemble_K(self, K: ti.types.sparse_matrix_builder()):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            for m, n in ti.static(ti.ndrange(2, 2)):
                K[2 * idx1 + m, 2 * idx1 + n] -= self.Jx[i][m, n]
                K[2 * idx1 + m, 2 * idx2 + n] += self.Jx[i][m, n]
                K[2 * idx2 + m, 2 * idx1 + n] += self.Jx[i][m, n]
                K[2 * idx2 + m, 2 * idx2 + n] -= self.Jx[i][m, n]


    @ti.kernel
    def assemble_D(self, D: ti.types.sparse_matrix_builder()):
        for i in self.spring:
            idx1, idx2 = self.spring[i][0], self.spring[i][1]
            for m, n in ti.static(ti.ndrange(2, 2)):
                D[2 * idx1 + m, 2 * idx1 + n] -= self.Jv[i][m, n]
                D[2 * idx1 + m, 2 * idx2 + n] += self.Jv[i][m, n]
                D[2 * idx2 + m, 2 * idx1 + n] += self.Jv[i][m, n]
                D[2 * idx2 + m, 2 * idx2 + n] -= self.Jv[i][m, n]


    @ti.kernel
    def updatePosVel(self, h: ti.f32, dv: ti.types.ndarray()):
        for i in self.pos:
            self.vel[i] += ti.Vector([dv[2 * i], dv[2 * i + 1]])
            self.pos[i] += h * self.vel[i]

    def update(self, h):
        self.compute_force()

        self.compute_Jacobians()
        # Assemble global system

        self.assemble_D(self.DBuilder)
        D = self.DBuilder.build()

        self.assemble_K(self.KBuilder)
        K = self.KBuilder.build()

        A = self.M - h * D - h**2 * K

        vel = self.vel.to_numpy().reshape(2 * self.NV)
        force = self.force.to_numpy().reshape(2 * self.NV)
        b = (force + h * K @ vel) * h
        # Sparse solver
        solver = ti.linalg.SparseSolver(solver_type="LDLT")
        solver.analyze_pattern(A)
        solver.factorize(A)
        # Solve the linear system
        dv = solver.solve(b)
        self.updatePosVel(h, dv)

    def display(self, gui, radius=5, color=0xffffff):
        lines = self.spring.to_numpy()
        pos = self.pos.to_numpy()
        edgeBegin = np.zeros(shape=(lines.shape[0], 2))
        edgeEnd = np.zeros(shape=(lines.shape[0], 2))
        for i in range(lines.shape[0]):
            idx1, idx2 = lines[i][0], lines[i][1]
            edgeBegin[i] = pos[idx1]
            edgeEnd[i] = pos[idx2]
        gui.lines(edgeBegin, edgeEnd, radius=2, color=0x0000ff)
        gui.circles(self.pos.to_numpy(), radius, color)

    @ti.kernel
    def spring2indices(self):
        for i in self.spring:
            self.indices[2 * i + 0] = self.spring[i][0]
            self.indices[2 * i + 1] = self.spring[i][1]

    def displayGGUI(self, canvas, radius=0.01, color=(1.0, 1.0, 1.0)):
        self.spring2indices()
        canvas.lines(self.pos,
                     width=0.005,
                     indices=self.indices,
                     color=(0.0, 0.0, 1.0))
        canvas.circles(self.pos, radius, color)


def main():
    ti.init(arch=ti.cpu)
    h = 0.01
    cloth = Cloth(N=5)

    pause = False
    parser = argparse.ArgumentParser()
    parser.add_argument('-g',
                        '--use-ggui',
                        action='store_true',
                        help='Display with GGUI')
    args, unknowns = parser.parse_known_args()
    use_ggui = False
    use_ggui = args.use_ggui

    if not use_ggui:
        gui = ti.GUI('Implicit Mass Spring System', res=(500, 500))
        while gui.running:
            for e in gui.get_events():
                if e.key == gui.ESCAPE:
                    gui.running = False
                elif e.key == gui.SPACE:
                    pause = not pause

            if not pause:
                cloth.update(h)

            cloth.display(gui)
            gui.show()
    else:
        window = ti.ui.Window('Implicit Mass Spring System', res=(500, 500))
        while window.running:
            if window.get_event(ti.ui.PRESS):
                if window.event.key == ti.ui.ESCAPE:
                    break
            if window.is_pressed(ti.ui.SPACE):
                pause = not pause

            if not pause:
                cloth.update(h)

            canvas = window.get_canvas()
            cloth.displayGGUI(canvas)
            window.show()


if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
